In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [ ]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed
from tmqrfeed.quotes.dataframegetter import DataFrameGetter

In [ ]:
feed = DataFeed()

In [ ]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2011, 1, 1))

In [ ]:
%load_ext line_profiler
import line_profiler
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [ ]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
cimport numpy as np
import cython
import numpy as np

DTYPE = np.float
ctypedef np.float64_t DTYPE_t
import pandas as pd


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def compress_daily(dfg, asset):
    """
    Calculate OHLCV based on 1-min data from PreProcessor
    :param dfg: DataFrameGetter instance
    :return:
    """

    cdef DTYPE_t _o, _h, _l, _c, _v, _exec_px

    npdate_buf = dfg.index.values.astype('datetime64[D]')

    cdef np.uint64_t[:] npdate = npdate_buf.view(np.uint64)
    cdef np.uint64_t[:] npdatetime = dfg.index.values.astype('datetime64[s]').view(np.uint64)

    cdef np.ndarray[DTYPE_t, ndim=2] data = dfg.data

    cdef int io = dfg.cols['o']
    cdef int ih = dfg.cols['h']
    cdef int il = dfg.cols['l']
    cdef int ic = dfg.cols['c']
    cdef int iv = dfg.cols['v']
    cdef int count = data.shape[1]
    cdef int i = 0
    cdef int exec_i = 0
    cdef np.uint64_t last_date = -1
    cdef int last_date_idx = -1
    values = []
    values_index = []

    exec_values = []
    exec_values_index = []

    # Session filter settings
    asset_session = asset.instrument_info.session
    cdef np.uint64_t sess_start = -1
    cdef np.uint64_t sess_decision = -1
    cdef np.uint64_t sess_execution = -1
    cdef np.uint64_t sess_next_date = -1
    
    cdef np.uint64_t current_date = -1
    cdef np.uint64_t current_datetime = -1
    
    dt_sess_start = dt_sess_decision = dt_sess_exec = dt_sess_next = None

    cdef int is_newday = 1 #

    for i in range(count):
        current_date = npdate[i]
        current_datetime = npdatetime[i]
        
        # If new day occurred
        if last_date != current_date:
            if not is_newday:
                # Store previous OHLCV values
                values.append(
                    {
                        'o': _o,
                        'h': _h,
                        'l': _l,
                        'c': _c,
                        'v': _v,
                        'exec': _exec_px,
                    }
                )
                values_index.append(dt_sess_decision)

                # Store exec values
                exec_values.append({
                    'date': dt_sess_decision,
                    'exec_time': dt_sess_exec,
                    'quote_time': asset_session.tz.localize(dfg.index[exec_i]),
                    'px': _exec_px,
                    'qty': 1,
                    'asset': asset,
                    }
                )

            # Calculate trading session params
            dt_sess_start, dt_sess_decision, dt_sess_exec, dt_sess_next = asset_session.get(dfg.index[i])
            sess_start = np.datetime64(dt_sess_start.replace(tzinfo=None)).astype('datetime64[s]').view(np.uint64)
            sess_decision = np.datetime64(dt_sess_decision.replace(tzinfo=None)).astype('datetime64[s]').view(np.uint64)
            sess_execution = np.datetime64(dt_sess_exec.replace(tzinfo=None)).astype('datetime64[s]').view(np.uint64)

            last_date = current_date
            last_date_idx = i
            is_newday = 1


        if current_datetime < sess_start or current_datetime >= sess_execution:
            continue



        if is_newday:
            _o = data[io, i]
            _h = data[ih, i]
            _l = data[il, i]
            _c = data[ic, i]
            _v = data[iv, i]
            _exec_px = _c
            exec_i = i
            is_newday = 0
        else:
            if current_datetime < sess_decision:
                _h = max(_h, data[ih, i])
                _l = min(_l, data[il, i])
                _c = data[ic, i]
                _v += data[iv, i]
                _exec_px = _c
                exec_i = i
            else:
                _exec_px = data[ic, i]
                exec_i = i

    if not is_newday:
        # Process last values
        values.append(
            {
                'o': _o,
                'h': _h,
                'l': _l,
                'c': _c,
                'v': _v,
                'exec': _exec_px,
            }
        )
        # Store exec values
        exec_values.append({
            'date': dt_sess_decision,
            'quote_time': asset_session.tz.localize(dfg.index[exec_i]),
            'exec_time': dt_sess_exec,
            'px': _exec_px,
            'qty': 1,
            'asset': asset,
        }
        )
        values_index.append(dt_sess_decision)

    df_result = pd.DataFrame(values, index=values_index)
    df_result.index.rename('dt', inplace=True)
    return df_result, pd.DataFrame(exec_values)


In [ ]:
fut = chain.get_contract(datetime(2012, 1, 1))
series = fut.get_series()
%timeit compress_daily(DataFrameGetter(series), fut)

In [ ]:
%lprun -f compress_daily compress_daily(DataFrameGetter(series), fut)

    

In [ ]:
fut = chain.get_contract(datetime(2012, 1, 1))
fut2 = chain.get_contract(datetime(2012, 1, 1), offset=1)

In [ ]:
series = fut.get_series()

In [ ]:
series.index

In [ ]:
series.index.tz_localize(None).values

In [ ]:
np.datetime64(datetime.now()).view('uint64').view('datetime64[us]')

In [ ]:
fut.instrument_info.session.sessions

In [ ]:
test_ser = series.ix['2011-12-20'].between_time('00:32', '10:39')

In [ ]:
res_ser = compress_daily(DataFrameGetter(series), fut).ix['2011-12-20']

In [ ]:
assert res_ser['o'] == test_ser['o'][0]
assert res_ser['c'] == test_ser['c'][-1]
assert res_ser['h'] == test_ser['h'].max()
assert res_ser['l'] == test_ser['l'].min()
assert res_ser['v'] == test_ser['v'].sum()

In [ ]:
from unittest.mock import MagicMock

In [ ]:
m = MagicMock()

In [ ]:
m.asset.info = 'x'

In [ ]:
m.asset.info

In [ ]:
def get_series():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(series.resample('D').apply(resampleer).dropna())    
        except:
            continue        
            

In [ ]:
def get_series_fast():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue        

In [ ]:
def get_series2():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue      
    return series, fut_contract

In [ ]:
ser, fut = get_series2()

In [ ]:
fut.instrument_info.session.get(ser.index[0], numpy_dtype=True)

In [ ]:
tz = pytz.timezone("US/Pacific")
datetime(2015, 7, 15, 18, 30, 12, tzinfo=tz)

In [ ]:
datetime.utcfromtimestamp(1437035100000000 / 1000000)

In [ ]:
compress_daily(DataFrameGetter(series), fut)

In [ ]:
np.datetime64(numpy.datetime64('2010-01-01T23:00:00'))

In [ ]:
compress_daily(fut.instrument_info.session.filter_dataframe(ser))

In [ ]:
%timeit get_series()

In [ ]:
%timeit get_series_fast()

In [ ]:
%lprun -f get_series get_series()

In [ ]:
%lprun -f get_series_fast get_series_fast()

In [ ]:
%lprun -m tmqrfeed.datafeed get_series()